In [102]:
import pandas as pd
import numpy as np
from Bio import Entrez, SeqIO, Align
import io

from IPython.display import HTML

# Data exploration

In [15]:
metadata = pd.read_csv("/kaggle/input/dataset/ncbi_datasets.csv")

/tmp/ipykernel_33/2690805399.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv("/kaggle/input/dataset/ncbi_datasets.csv")


In [7]:
metadata

,Nucleotide Accession,Species Taxonomy Id,Species Name,Virus Genus,Virus Family,Isolate Name,Nucleotide Length,Sequence Type,Nuc Completeness,Geo Location,US State,Host Name,Host Taxonomy ID,Collection Date,BioProject,BioSample
0,NC_045512.2,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,Wuhan-Hu-1,29903,RefSeq,complete,Asia; China,NaN,NaN,9606,2019-12,PRJNA485481,NaN
1,OM487257.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/UT-UPHL-220129247957/2021,29815,GenBank,complete,North America; USA,NaN,NaN,9606,2021-12-23,PRJNA614995,SAMN25559487
2,OM403304.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/HKG/VM20001061-2/2020,29870,GenBank,complete,Asia; Hong Kong,NaN,NaN,9606,2020-01-23,NaN,NaN
3,OM403303.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/HKG/Original/2020,29851,GenBank,complete,Asia; Hong Kong,NaN,NaN,9606,2020-01-23,NaN,NaN
4,OM401120.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/UT-UPHL-211112790633/2021,29712,GenBank,complete,North America; USA,NaN,NaN,9606,2021-09-25,PRJNA614995,SAMN23427251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847786,HG996822.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,a925,29903,GenBank,complete,Europe; United Kingdom:Oxford,NaN,NaN,9606,2020-11-27,PRJEB43319,SAMEA8246553
847787,HG996820.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,a935,29903,GenBank,complete,Europe; United Kingdom:Oxford,NaN,NaN,9606,2020-12-10,PRJEB43319,SAMEA8246563
847788,HG996813.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,a929,29903,GenBank,complete,Europe; United Kingdom:Oxford,NaN,NaN,9606,2020-12-01,PRJEB43319,SAMEA8246557
847789,HG996715.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,a850,29897,GenBank,complete,Europe; United Kingdom:Oxford,NaN,NaN,9606,2020-11-29,PRJEB43319,SAMEA8246478


In [8]:
print("The shape of metadata-Df is: ", metadata.shape)

The shape of metadata-Df is:  (847791, 16)


In [17]:
metadata.dtypes

Nucleotide Accession     object
Species Taxonomy Id       int64
Species Name             object
Virus Genus              object
Virus Family             object
Isolate Name             object
Nucleotide Length         int64
Sequence Type            object
Nuc Completeness         object
Geo Location             object
US State                float64
Host Name               float64
Host Taxonomy ID          int64
Collection Date          object
BioProject               object
BioSample                object
dtype: object

In [18]:
metadata.columns = [c.lower().replace(" ","_") for c in metadata.columns]

In [21]:
# Get column names as a list
column_names = metadata.columns.tolist()
column_names

['nucleotide_accession',
 'species_taxonomy_id',
 'species_name',
 'virus_genus',
 'virus_family',
 'isolate_name',
 'nucleotide_length',
 'sequence_type',
 'nuc_completeness',
 'geo_location',
 'us_state',
 'host_name',
 'host_taxonomy_id',
 'collection_date',
 'bioproject',
 'biosample']

In [22]:
metadata["continent"] = metadata["geo_location"].str.replace(";.+","", regex=True)

In [40]:
metadata.head(10)

,nucleotide_accession,species_taxonomy_id,species_name,virus_genus,virus_family,isolate_name,nucleotide_length,sequence_type,nuc_completeness,geo_location,us_state,host_name,host_taxonomy_id,collection_date,bioproject,biosample,continent
0,NC_045512.2,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,Wuhan-Hu-1,29903,RefSeq,complete,Asia; China,NaN,NaN,9606,2019-12,PRJNA485481,NaN,Asia
1,OM487257.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/UT-UPHL-220129247957/2021,29815,GenBank,complete,North America; USA,NaN,NaN,9606,2021-12-23,PRJNA614995,SAMN25559487,North America
2,OM403304.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/HKG/VM20001061-2/2020,29870,GenBank,complete,Asia; Hong Kong,NaN,NaN,9606,2020-01-23,NaN,NaN,Asia
3,OM403303.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/HKG/Original/2020,29851,GenBank,complete,Asia; Hong Kong,NaN,NaN,9606,2020-01-23,NaN,NaN,Asia
4,OM401120.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/UT-UPHL-211112790633/2021,29712,GenBank,complete,North America; USA,NaN,NaN,9606,2021-09-25,PRJNA614995,SAMN23427251,North America
5,OM389146.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/GA-EHC-117N/2020,29790,GenBank,complete,North America; USA,NaN,NaN,9606,2020-03-30,PRJNA634356,SAMN16129847,North America
6,OM389144.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/GA-EHC-111G/2020,29797,GenBank,complete,North America; USA,NaN,NaN,9606,2020-03-30,PRJNA634356,SAMN16129844,North America
7,OM389143.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/GA-EHC-087I/2020,29817,GenBank,complete,North America; USA,NaN,NaN,9606,2020-03-10,PRJNA634356,SAMN24965877,North America
8,OM389139.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/GA-EHC-060H/2020,29824,GenBank,complete,North America; USA,NaN,NaN,9606,2020-03-11,PRJNA634356,SAMN24965874,North America
9,OM389137.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/GA-EHC-047U/2020,29882,GenBank,complete,North America; USA,NaN,NaN,9606,2020-03-15,PRJNA634356,SAMN24965870,North America


In [24]:
## Get the first COVID-sequence, that was sequenced on each continent

metadata.groupby("continent").apply(lambda x: x.sort_values("collection_date").iloc[0])

/tmp/ipykernel_33/2248045959.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metadata.groupby("continent").apply(lambda x: x.sort_values("collection_date").iloc[0])


,nucleotide_accession,species_taxonomy_id,species_name,virus_genus,virus_family,isolate_name,nucleotide_length,sequence_type,nuc_completeness,geo_location,us_state,host_name,host_taxonomy_id,collection_date,bioproject,biosample,continent
continent,,,,,,,,,,,,,,,,,
Africa,OL336607.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/MLI/M00960/2020,29890,GenBank,complete,Africa; Mali,NaN,NaN,9606,2020-02-06,NaN,NaN,Africa
Asia,NC_045512.2,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,Wuhan-Hu-1,29903,RefSeq,complete,Asia; China,NaN,NaN,9606,2019-12,PRJNA485481,NaN,Asia
Europe,OU393175.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,NaN,29890,GenBank,complete,Europe; United Kingdom:England,NaN,NaN,9606,2020,PRJEB37886,SAMEA9354774,Europe
North America,OL467832.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/hCoV_19_USA_ID_IVREF_6520...,29831,GenBank,complete,"North America; USA: Bingham, ID",NaN,NaN,9606,2020,NaN,NaN,North America
Oceania,MW320919.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/AUS/VIC16874/2020,29790,GenBank,complete,Oceania; Australia: Victoria,NaN,NaN,9606,2020,NaN,SAMN16975285,Oceania
South America,MZ477858.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/BRA/PR-HF441/2020,29779,GenBank,complete,South America; Brazil: Parana,NaN,NaN,9606,2020,NaN,NaN,South America


In [25]:
# Nucleotide length

metadata["nucleotide_length"].describe()

count    847791.000000
mean      29825.747932
std          69.937482
min        2859.000000
25%       29786.000000
50%       29819.000000
75%       29884.000000
max       30018.000000
Name: nucleotide_length, dtype: float64

# Identifying sequences to download

In [28]:
# Define Reference Seq

metadata[metadata['sequence_type'] == "RefSeq"]

,nucleotide_accession,species_taxonomy_id,species_name,virus_genus,virus_family,isolate_name,nucleotide_length,sequence_type,nuc_completeness,geo_location,us_state,host_name,host_taxonomy_id,collection_date,bioproject,biosample,continent
0,NC_045512.2,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,Wuhan-Hu-1,29903,RefSeq,complete,Asia; China,NaN,NaN,9606,2019-12,PRJNA485481,NaN,Asia


In [36]:
##
metadata["isolate_name"] = metadata["isolate_name"].astype(str)

In [38]:
## Filter out the "Delta" Variant sequences

metadata[metadata["isolate_name"].str.contains("Delta").fillna(False)]

,nucleotide_accession,species_taxonomy_id,species_name,virus_genus,virus_family,isolate_name,nucleotide_length,sequence_type,nuc_completeness,geo_location,us_state,host_name,host_taxonomy_id,collection_date,bioproject,biosample,continent
85804,OM108163.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/CHN/Delta-3/2021,29860,GenBank,complete,Asia; China: Beijing,NaN,NaN,9606,2021-08-14,NaN,NaN,Asia
85948,OM061695.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/CHN/Delta-1/2021,29858,GenBank,complete,Asia; China: Beijing,NaN,NaN,9606,2021-08-10,NaN,NaN,Asia
251375,OK091006.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,"SARS-CoV-2/human/JPN/SARS-CoV-2, B.1.617.2 lin...",29836,GenBank,complete,Asia; Japan,NaN,NaN,9606,2021-08-24,NaN,NaN,Asia
665095,OM108132.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/CHN/Delta-2/2021,29858,GenBank,complete,Asia; China: Beijing,NaN,NaN,9606,2021-08-11,NaN,NaN,Asia


In [39]:
## Filter out the "Omicron" Variant sequences

metadata[metadata["isolate_name"].str.contains("Omicron").fillna(False)]

,nucleotide_accession,species_taxonomy_id,species_name,virus_genus,virus_family,isolate_name,nucleotide_length,sequence_type,nuc_completeness,geo_location,us_state,host_name,host_taxonomy_id,collection_date,bioproject,biosample,continent
806923,OM095411.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/CHN/Omicron-1/2021,29788,GenBank,complete,Asia; China: Beijing,NaN,NaN,9606,2021-12-08,NaN,NaN,Asia


In [51]:
## A list with "Nuclotide Acession numbers" 
## of the Ref-seq., another not mutated sequence and a "Delta" variant as well as an "omicron" sequence var

sequences = ["NC_045512.2", "OM487257.1", "OM061695.1", "OM095411.1"]
human_names = ["reference", "base", "delta", "omicron"]

In [52]:
selected_sequences = metadata[metadata['nucleotide_accession'].isin(sequences)]

In [53]:
selected_sequences

,nucleotide_accession,species_taxonomy_id,species_name,virus_genus,virus_family,isolate_name,nucleotide_length,sequence_type,nuc_completeness,geo_location,us_state,host_name,host_taxonomy_id,collection_date,bioproject,biosample,continent
0,NC_045512.2,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,Wuhan-Hu-1,29903,RefSeq,complete,Asia; China,NaN,NaN,9606,2019-12,PRJNA485481,NaN,Asia
1,OM487257.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/UT-UPHL-220129247957/2021,29815,GenBank,complete,North America; USA,NaN,NaN,9606,2021-12-23,PRJNA614995,SAMN25559487,North America
85948,OM061695.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/CHN/Delta-1/2021,29858,GenBank,complete,Asia; China: Beijing,NaN,NaN,9606,2021-08-10,NaN,NaN,Asia
806923,OM095411.1,2697049,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/CHN/Omicron-1/2021,29788,GenBank,complete,Asia; China: Beijing,NaN,NaN,9606,2021-12-08,NaN,NaN,Asia


In [54]:
##  Function to get the wanted Sequences based on the selecte accession numbers

def download_sequence(id_code):
    handle = Entrez.esearch(db="nucleotide", term=id_code, retmax="1")
    record = Entrez.read(handle)
    handle = Entrez.efetch(db="nucleotide", id=record["IdList"][0], rettype="fasta", retmode="text")
    return handle.read()
    

In [55]:
## Get and save the sequences in fasta format in a dictionary "ID - SEQUENCE"

sequence_data = {}

for seq in sequences:
    sequence_data[seq] = {"fasta": download_sequence(seq)}

In [61]:
## First COVID- sequence
print(sequence_data["NC_045512.2"])

{'fasta': '>NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome\...\nAAAAAAAAAAAAA\n\n'}


## PARSING RNA Sequences

In [65]:
## Using the FASTA - parser in biopython

for n_id, seq in sequence_data.items():
    temp_file = io.StringIO(seq["fasta"])
    sequence_data[n_id]["parsed"] = list(SeqIO.parse(temp_file, "fasta"))[0]

In [66]:
sequence_data["NC_045512.2"]["parsed"]

SeqRecord(seq=Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA'), id='NC_045512.2', name='NC_045512.2', description='NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome', dbxrefs=[])

# ALIGNING RNA SEQUENCES

In [68]:
## Using the "Align" methode from biopython that uses NMW-alg

aligner = Align.PairwiseAligner()

In [69]:
aligner.algorithm

'Needleman-Wunsch'

In [76]:
score_1 = aligner.score(sequence_data["NC_045512.2"]["parsed"].seq, sequence_data["OM487257.1"]["parsed"].seq)

In [78]:
score_1

29791.0

In [79]:
seq_length = len(sequence_data["NC_045512.2"]["parsed"].seq)
print(seq_length)

29903


In [82]:
##  Compare Ref-Sequence with the coded seq. of the first variant in north America
print('{:.2f}%'.format((score_1/seq_length) * 100))

99.63%


## ALIGNMENTS OF ALL CORONA VARIANTS

In [84]:
##  First create a Comparisons-Matrix, initiated with 0s

comparisons_mtx = np.zeros([4,4])

In [85]:
comparisons_mtx

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [89]:
## ALIGN THE SEQUENCES

for i in range(0,4):
    for j in range(0,i+1):
        score = aligner.score(sequence_data[sequences[i]]["parsed"].seq, sequence_data[sequences[j]]["parsed"].seq)
        comparisons_mtx[i,j] = score

In [90]:
comparisons_df = pd.DataFrame(comparisons_mtx, columns=human_names, index=human_names)

In [91]:
comparisons_df

,reference,base,delta,omicron
reference,29903.0,0.0,0.0,0.0
base,29791.0,29815.0,0.0,0.0
delta,29818.0,29752.0,29858.0,0.0
omicron,29725.0,29711.0,29688.0,29788.0


In [92]:
comparisons_df.iloc[:,0] / seq_length

reference    1.000000
base         0.996255
delta        0.997157
omicron      0.994047
Name: reference, dtype: float64

## MUTATION POINTS

In [93]:
## Compare

ref_seq = sequence_data["NC_045512.2"]["parsed"].seq
omicron_seq = sequence_data["OM095411.1"]["parsed"].seq

omicron_alignments = aligner.align(ref_seq, omicron_seq)

In [96]:
omicron_alignments

In [97]:
omicron_alignment = omicron_alignments[0]

In [99]:
omicron_alignment.aligned

array([[[   14,    15],
        [   21,    22],
        [   25,    26],
        [   27,    28],
        [   40,   240],
        [  241,  2469],
        [ 2469,  2471],
        [ 2472,  2831],
        [ 2832,  2833],
        [ 2833,  3036],
        [ 3037,  3038],
        [ 3038,  5385],
        [ 5386,  5387],
        [ 5387,  6512],
        [ 6513,  6514],
        [ 6516,  8392],
        [ 8393, 10028],
        [10028, 10029],
        [10030, 10448],
        [10448, 10449],
        [10450, 11286],
        [11287, 11288],
        [11296, 11536],
        [11537, 13194],
        [13195, 14407],
        [14408, 14409],
        [14409, 15239],
        [15240, 18162],
        [18163, 21761],
        [21762, 21763],
        [21764, 21766],
        [21768, 21769],
        [21771, 21845],
        [21846, 21847],
        [21847, 21986],
        [21992, 21993],
        [21996, 22193],
        [22194, 22195],
        [22197, 22206],
        [22206, 22577],
        [22578, 22598],
        [22599, 

In [103]:
#

def color_print(s, color="black"):
    return "<span style='color:{}'>{}</span>".format(color, s)

In [125]:
##
##

ref_seq_end = None
omicron_seq_end= None
display_list = []

for alignments in zip(omicron_alignment.aligned[0], omicron_alignment.aligned[1]):
    if ref_seq_end and omicron_seq_end:
        
        ref_seq_mismatch = ref_seq[ref_seq_end: alignments[0][0]]
        omicron_seq_mismatch = omicron_seq[omicron_seq_end: alignments[0][0]]
        
        # Deletion
        if len(omicron_seq_mismatch) == 0:
            display_list.append(color_print(ref_seq_mismatch, "red"))
        # Insertion
        elif len(ref_seq_mismatch) == 0:
            display_list.append(color_print(omicron_seq_mismatch, "green"))
        # Substitution
        else:
            display_list.append(color_print(omicron_seq_mismatch, "blue"))

    display_list.append(ref_seq[alignments[0][0]:alignments[0][1]])
        #print("1: {}".format(ref_seq_mismatch))
        #print("2: {}".format(omicron_seq_mismatch))
        
    ref_seq_end = alignments[0][1]
    omicron_seq_end = alignments[1][1]


In [126]:
display_list = [str(i) for i in display_list]

In [127]:
display(HTML('<br>'.join(display_list)))